<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow and Keras
Tensorflow is a deep learning framewok that supports all the mathematical operations on tensors, provides the basic components to build a neural network such as different kind of layers and activation functions, optimizers, and supports the automatic differentiation to perform the backpropagation algorithm.

In [3]:
import tensorflow as tf

In [5]:
x = tf.ones(shape=(2, 1))
x

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[1.],
       [1.]], dtype=float32)>